In [7]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = ''


In [8]:
from transformers import AutoModelWithLMHead, AutoTokenizer

path = "../experiments/model/t5-base-finetuned-wikiSQL"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelWithLMHead.from_pretrained(path)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/home/fengyuan/anaconda3/envs/transformers/lib/python3.6/site-packages/transformers/models/auto/modeling_auto.py:664: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [9]:
def get_sql(query):
  input_text = "translate English to SQL: %s </s>" % query
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'])
  
  return tokenizer.decode(output[0])

In [10]:
query = "How many models were finetuned using BERT as base model?"

get_sql(query)

'<pad> SELECT COUNT Model fine tuned FROM table WHERE Base model = BERT</s>'

In [6]:
query = "How many people are there is Shanghai?"

get_sql(query)

'<pad> SELECT COUNT Population FROM table WHERE City = Shanghai</s>'

In [15]:

query = "How many people in Shanghai are older than 50?"

get_sql(query)

'<pad> SELECT COUNT Number FROM table WHERE Age > 50 AND City = s'

In [22]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelWithLMHead


class Text2SQL:

    def __init__(self, t5_path, mt_path):
        self.tokenizer_t5 = AutoTokenizer.from_pretrained(t5_path)
        self.model_t5 = AutoModelWithLMHead.from_pretrained(t5_path)
        self.tokenizer_mt = AutoTokenizer.from_pretrained(mt_path)
        self.model_mt = AutoModelForSeq2SeqLM.from_pretrained(mt_path)

    def zh2en(self, sentence):
        self.tokenizer_mt.padding_side = "left"
        self.tokenizer_mt.pad_token = self.tokenizer_mt.eos_token # to avoid an error
        task_prefix = 'translate Chinese to English: '
        inputs = self.tokenizer_mt(task_prefix + sentence, return_tensors="pt", padding=True)

        output_sequences = self.model_mt.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            do_sample=False, # disable sampling to test if batching affects output
        )
   

        result = self.tokenizer_mt.batch_decode(output_sequences, skip_special_tokens=True)[0]
        result = result.replace('<pad>','')
        result = result.strip()
        return result

    def text2sql(self, query):
        input_text = "translate English to SQL: %s </s>" % query
        features = self.tokenizer_t5([input_text], return_tensors='pt')
        output = self.model_t5.generate(input_ids=features['input_ids'], attention_mask=features['attention_mask'])
        result = self.tokenizer_t5.decode(output[0])
        result = result.replace('<pad>','')
        result = result.strip()
        return result

    def do_inference(self, input_sent, table_id):
        result = self.zh2en(input_sent)
        result = self.text2sql(result)
        result = result.replace('</s>','')
        result = result.replace('table', table_id)
        return result

t5_path =  "../experiments/model/t5-base-finetuned-wikiSQL"
mt_path =  "../experiments/model/opus-mt-zh-en"
text2sql = Text2SQL(t5_path,mt_path)


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/home/fengyuan/anaconda3/envs/transformers/lib/python3.6/site-packages/transformers/models/auto/modeling_auto.py:664: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [23]:
query = '上海有多少人口？'
table_id = '123124'

print(text2sql.do_inference(query, table_id))

SELECT COUNT Population FROM 123124 WHERE City = Shanghai


In [24]:
query = '美国最大的城市是哪个？'
table_id = 'fsadf'

print(text2sql.do_inference(query, table_id))

SELECT MAX City FROM fsadf WHERE Country = united states
